In [76]:
import pandas as pd

In [77]:
%run TableCreationBaseline.py
%run addMidruleToLatex.py

In [78]:
year = 2017
df = pd.read_csv("/Users/faessler/nfshome/Research/sigir/pmclass/pmclass_analysis"+str(year)+".csv").round(2)
df["year"] = pd.Series([year]*len(df), index=df.index)

In [79]:
bltable = getBaselineDf()
blline = bltable.query("year == " + str(year)).drop(["P_10", "year"], axis=1).stack()
blline.index = blline.index.droplevel()

# First, which template won, especially must vs. should

In [80]:
df.query("not classifier.str.contains('"+str(year)+"')")
best = df.loc[df["infNDCG"].idxmax()]

In [81]:
df.query("not classifier.str.contains('"+str(year)+"')")

,template,clausetype,op,classifier,recall,relgs,relfound,ndcg,infNDCG,Rprec,P_10,mean_r,std_r,year
2,all_must,best_fields,OR,pmclass2018.keyword,63.10,3875,2445,32.10,24.47,14.68,29.33,1306.21,1823.67,2017
3,all_must,best_fields,OR,pmclass2018lstmgru.keyword,61.50,3875,2383,32.65,24.75,15.38,30.67,1210.93,1726.72,2017
6,all_must,phrase,OR,pmclass2018.keyword,49.11,3875,1903,44.61,43.04,27.46,54.33,326.34,596.31,2017
7,all_must,phrase,OR,pmclass2018lstmgru.keyword,48.39,3875,1875,42.61,41.28,25.78,53.00,333.64,582.71,2017
10,all_should,best_fields,OR,pmclass2018.keyword,82.25,3875,3187,36.59,24.91,15.02,31.00,1748.99,2229.43,2017
11,all_should,best_fields,OR,pmclass2018lstmgru.keyword,82.55,3875,3199,37.14,25.36,15.47,31.00,1706.92,2179.66,2017
14,all_should,phrase,OR,pmclass2018.keyword,67.30,3875,2608,54.56,46.14,30.33,56.67,498.05,866.20,2017
15,all_should,phrase,OR,pmclass2018lstmgru.keyword,67.30,3875,2608,54.50,46.15,30.03,56.33,497.18,860.78,2017
18,baseline_must,best_fields,OR,pmclass2018.keyword,65.39,3875,2534,41.46,32.46,21.76,41.33,728.19,1121.75,2017
19,baseline_must,best_fields,OR,pmclass2018lstmgru.keyword,63.48,3875,2460,41.40,32.53,21.94,42.00,677.18,1083.86,2017


# And give the complete mean values over the better template

In [82]:
blline

recall       83.28
infNDCG      38.83
Rprec        27.42
mean_r      763.16
std_r      1182.27
dtype: float64

In [83]:
def mapindex(tup):
    m = {"all_must":"and, all", "all_should":"or, all","baseline_must": "and, base", "baseline_should": "or, base", "pmclass2018.keyword": "LogReg", "pmclass2018lstmgru.keyword": "DL", "baseline": "baseline", "":"",  "pmclass2017.keyword": "LogReg", "pmclass2017lstmgru.keyword": "DL",}
    tmp = tup[0]
    cls = tup[1]
    return (m[tmp], m[cls])
table = df.query("not classifier.str.contains('"+str(year)+"')")
table = table.groupby(["template", "classifier"]).mean().reset_index()
table = table.append(best)
table = table.drop(["relgs", "relfound", "ndcg", "P_10", "year", "clausetype", "op", "Rprec"], axis=1)
table = table.reset_index()
table.loc["baseline"] = blline
table["template"].iat[-1] = "baseline"
table["classifier"].iat[-1] = ""
table = table.set_index(["template", "classifier"])
table.index = table.index.map(mapindex)
table
table.index.names = ["op", "model"]
table = table.drop("index", axis=1)
table
latex = table.round(1).to_latex()
addMidruleToLatex(latex)

\begin{tabular}{llrrrr}
\toprule
         &    &  recall &  infNDCG &  mean\_r &   std\_r \\
op & model &         &          &         &         \\
\midrule
and, all & LogReg &    56.1 &     33.8 &   816.3 &  1210.0 \\
         & DL &    54.9 &     33.0 &   772.3 &  1154.7 \\
or, all & LogReg &    74.8 &     35.5 &  1123.5 &  1547.8 \\
         & DL &    74.9 &     35.8 &  1102.0 &  1520.2 \\
and, base & LogReg &    57.2 &     37.8 &   529.2 &   863.2 \\
         & DL &    55.9 &     36.8 &   507.0 &   835.3 \\
or, base & LogReg &    79.4 &     39.7 &   832.7 &  1255.6 \\
         & DL &    79.5 &     39.7 &   828.1 &  1253.4 \\
or, all & DL &    67.3 &     48.4 &   465.4 &   803.8 \\
\midrule
baseline &    &    83.3 &     38.8 &   763.2 &  1182.3 \\
\bottomrule
\end{tabular}


In [84]:
pd.concat([table, best],axis=1)

Exception: cannot handle a non-unique multi-index!

In [ ]:
best